Data Engineering 

In [6]:
import pandas as pd
import numpy as np
from datetime import  datetime
import os 

In [68]:
archivos = ['amazon_prime_titles-score.csv','disney_plus_titles-score.csv','hulu_titles-score (2).csv','netflix_titles-score.csv']
os.makedirs('Datasets', exist_ok=True)
for x in archivos:
    Dataset = pd.read_csv('Datasets/'+ x)
    
    Dataset['duration_int'] = 0 #El campo duration debe convertirse en dos campos: duration_int y duration_type.
    Dataset['duration_type'] = 0 #El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)
    
    meses = ['January','February','March','April','May','June','July','August','September','October','November','December']
    for index,serie in Dataset.iterrows():
        Dataset.iat[index,0] = x[0] + serie[0] #Cada id se compondrá de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets
        fecha = str(Dataset.iat[index,6])
        Dataset.iat[index,1] = str(serie[1]).lower() #Los campos de texto deberán estar en minúsculas
        Dataset.iat[index,2] = str(serie[2]).lower() #Los campos de texto deberán estar en minúsculas
        Dataset.iat[index,3] = str(serie[3]).lower() #Los campos de texto deberán estar en minúsculas
        Dataset.iat[index,4] = str(serie[4]).lower() #Los campos de texto deberán estar en minúsculas
        Dataset.iat[index,5] = str(serie[5]).lower() #Los campos de texto deberán estar en minúsculas
        Dataset.iat[index,10] = str(serie[10]).lower() #Los campos de texto deberán estar en minúsculas
        Dataset.iat[index,11] = str(serie[11]).lower() #Los campos de texto deberán estar en minúsculas
        if serie[2][0] == '#':
                 Dataset.iat[index,2] = str(serie[2]).replace('#','')
        if serie[2][0] == '"':
                 Dataset.iat[index,2] = str(serie[2]).replace('"','')
        if serie[2][0] == "'":
                 Dataset.iat[index,2] = str(serie[2]).replace("'",'')
        if str(Dataset.iat[index,8]) == 'nan':
            Dataset.iat[index,8] = 'G' #Los valores nulos del campo rating deberán reemplazarse por el string “G”
        if len(str(serie[8]).split()) >= 2:
            if str(serie[8]).split()[1] in ['Season','min','Seasons']:
                Dataset.iat[index,13] = str(serie[8]).split()[0]
                Dataset.iat[index,14] = str(serie[8]).split()[1][:6].lower()
                Dataset.iat[index,8] = 'G' #Los valores nulos del campo rating deberán reemplazarse por el string “G”
        if fecha.split()[0] in meses:
            # las fechas, deberán tener el formato AAAA-mm-dd
            Dataset.iat[index,6] = datetime.strptime((fecha.split()[2] + '/' + str(meses.index(fecha.split()[0]) + 1) + '/' + fecha.split()[1].replace(',','')),'%Y/%m/%d')
        if str(serie[9]).lower() == 'nan':
            Dataset.iat[index,13] = 0
            Dataset.iat[index,14] = 'sin dato'
        else:
            Dataset.iat[index,13] = str(serie[9]).split()[0]  #todo el texto que esta en la columna duration pasa a las dos columnas creadas
            Dataset.iat[index,14] = str(serie[9]).split()[1][:6].lower()
    Nuevo_Dataset = Dataset.drop(['duration'],axis=1)
    if archivos.index(x) >= 1:
        Nuevo_Dataset.to_csv('Datasets/out.csv',header=False,index=None,mode='a')
    else:
        Nuevo_Dataset.to_csv('Datasets/out.csv',index=None,mode='a')


In [69]:
df = pd.read_csv('Datasets/out.csv')
identificador = {'amazon':'a','disney':'d','hulu':'h','netflix':'n'}
df.groupby('duration_type').count()

,show_id,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,score,duration_int
duration_type,,,,,,,,,,,,,
Sin dato,482,482,482,3,3,302,482,482,482,482,479,482,482
min,15999,15999,15999,14506,13636,7565,8201,15999,15999,15999,15999,15999,15999
season,6517,6517,6517,230,4038,3632,4761,6517,6517,6517,6516,6517,6517


In [56]:
#Cantidad de veces que aparece una keyword en el título de peliculas/series, por plataforma
def get_word_count(plataforma,keyword):
    count = 0
    for index,serie in df.iterrows():
        if str(keyword) in serie[2] and identificador[str(plataforma).lower()] in serie[0]:
            count += 1
    print(count)
get_word_count('netflix','love')

196


In [57]:
#Cantidad de películas por plataforma con un puntaje mayor a XX en determinado año
def  get_score_count(plataforma,score,año):
    count = 0
    for index,serie in df.iterrows():
        if identificador[str(plataforma).lower()] in serie[0] and serie[11] > score and serie[7] == año:
            count += 1
    print(count)
get_score_count('netflix',85,2010)

25


In [58]:
#La segunda película con mayor score para una plataforma determinada, según el orden alfabético de los títulos.
segunda_película = df.sort_values('title')
def get_second_score(plataforma):
    count = 0
    for index,serie in segunda_película.iterrows():
        if identificador[str(plataforma).lower()] in serie[0] and serie[1] == 'movie' and serie[11] >= 100:
            count += 1
            if count == 2:
                print(serie[2],' ',serie[11])
                break
get_second_score('amazon')

abilene town   100


In [84]:
#Película que más duró según año, plataforma y tipo de duración
def get_longest(plataforma,tipo,año):
    count = {}
    for index,serie in df.iterrows():
        if serie[7] == año and identificador[str(plataforma).lower()] in serie[0] and serie[13] == str(tipo).lower():
            count[serie[12]] = serie[2]
    print(count[sorted(count.keys()).pop()],' ',sorted(count.keys()).pop())
get_longest('amazon','min',2010)


band baaja baaraat   139


In [61]:
#Cantidad de series y películas por rating
def get_rating_count(rating):
    count = 0
    for index,serie in df.iterrows():
        if rating in serie[8]:
            count += 1
    print(count)
get_rating_count('TV-Y7')

569
